# Azure AI Agents với Hỗ trợ Giao thức Ngữ cảnh Mô hình (MCP) - Python

Notebook này minh họa cách sử dụng Azure AI Agents với các công cụ Giao thức Ngữ cảnh Mô hình (MCP) trong Python. Nó hướng dẫn cách tạo một agent thông minh có thể tận dụng các máy chủ MCP bên ngoài (như Microsoft Learn) để nâng cao khả năng bằng cách sử dụng xác thực không cần khóa.


## Cài đặt các gói Python cần thiết

Đầu tiên, chúng ta cần cài đặt các gói Python cần thiết:
- **azure-ai-projects**: SDK chính cho Azure AI Projects
- **azure-ai-agents**: SDK Azure AI Agents để tạo và quản lý các agent
- **azure-identity**: Cung cấp xác thực không cần khóa bằng DefaultAzureCredential
- **mcp**: Triển khai giao thức Model Context Protocol cho Python


## Lợi ích của Xác thực Không Dùng Khóa

Notebook này trình bày **xác thực không dùng khóa**, mang lại nhiều lợi ích:
- ✅ **Không cần quản lý API keys** - Sử dụng xác thực dựa trên danh tính Azure
- ✅ **Tăng cường bảo mật** - Không lưu trữ thông tin bí mật trong mã hoặc tệp cấu hình
- ✅ **Tự động xoay vòng thông tin xác thực** - Azure quản lý vòng đời của thông tin xác thực
- ✅ **Kiểm soát truy cập dựa trên vai trò** - Sử dụng Azure RBAC để phân quyền chi tiết
- ✅ **Hỗ trợ đa môi trường** - Hoạt động mượt mà giữa môi trường phát triển và sản xuất

`DefaultAzureCredential` tự động chọn nguồn thông tin xác thực tốt nhất hiện có:
1. **Managed Identity** (khi chạy trong Azure)
2. Thông tin xác thực từ **Azure CLI** (trong quá trình phát triển cục bộ)
3. Thông tin xác thực từ **Visual Studio**
4. **Biến môi trường** (nếu được cấu hình)
5. Xác thực qua **trình duyệt tương tác** (dùng làm phương án dự phòng)


## Cài đặt Xác thực Không cần Khóa

**Yêu cầu trước khi sử dụng xác thực không cần khóa:**

### Đối với Phát triển Cục bộ:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Đối với Môi trường Azure:
- Kích hoạt **Managed Identity được gán bởi hệ thống** trên tài nguyên Azure của bạn
- Gán các **vai trò RBAC** phù hợp cho managed identity:
  - `Cognitive Services OpenAI User` để truy cập Azure OpenAI
  - `AI Developer` để truy cập các Dự án AI của Azure

### Biến Môi trường (Tùy chọn):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Không cần khóa API hay chuỗi kết nối!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Nhập Các Thư Viện Cần Thiết

Nhập các mô-đun Python cần thiết:  
- **os, time**: Thư viện tiêu chuẩn của Python để làm việc với biến môi trường và độ trễ  
- **AIProjectClient**: Khách hàng chính cho các Dự án AI của Azure  
- **DefaultAzureCredential**: Xác thực không cần khóa cho các dịch vụ Azure  
- **Các lớp liên quan đến MCP**: Dùng để tạo và quản lý công cụ MCP và xử lý phê duyệt  


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Cấu hình Cài đặt Máy chủ MCP

Thiết lập cấu hình máy chủ MCP bằng cách sử dụng các biến môi trường với giá trị mặc định dự phòng:
- **MCP_SERVER_URL**: URL của máy chủ MCP (mặc định là Microsoft Learn API)
- **MCP_SERVER_LABEL**: Nhãn để nhận diện máy chủ MCP (mặc định là "mslearn")

Cách tiếp cận này cho phép cấu hình linh hoạt trên các môi trường khác nhau.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Tạo Client Dự Án Azure AI (Xác Thực Không Dùng Khóa)

Khởi tạo client dự án Azure AI sử dụng **xác thực không dùng khóa**:
- **endpoint**: URL endpoint của dự án Azure AI Foundry
- **credential**: Sử dụng `DefaultAzureCredential()` để xác thực an toàn, không cần khóa
- **Không cần khóa API**: Tự động phát hiện và sử dụng thông tin xác thực tốt nhất hiện có

**Quy trình Xác Thực:**
1. Kiểm tra Managed Identity (trong môi trường Azure)
2. Sử dụng thông tin xác thực từ Azure CLI (cho phát triển cục bộ)
3. Sử dụng các nguồn thông tin xác thực khác nếu cần

Cách tiếp cận này loại bỏ nhu cầu quản lý khóa API hoặc chuỗi kết nối trong mã của bạn.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Tạo Định Nghĩa Công Cụ MCP

Tạo một công cụ MCP kết nối với máy chủ MCP của Microsoft Learn:
- **server_label**: Nhãn định danh cho máy chủ MCP
- **server_url**: Điểm cuối URL của máy chủ MCP
- **allowed_tools**: Danh sách tùy chọn để giới hạn các công cụ có thể sử dụng (danh sách trống cho phép tất cả các công cụ)

Công cụ này sẽ giúp tác nhân truy cập tài liệu và tài nguyên của Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Tạo Agent và Thực Hiện Cuộc Hội Thoại (Quy Trình Không Dùng Khóa)

Phần hướng dẫn toàn diện này trình bày quy trình **agent không dùng khóa** hoàn chỉnh:

1. **Tạo AI Agent**: Thiết lập một agent với mô hình GPT-4.1 nano và các công cụ MCP
2. **Tạo Thread**: Khởi tạo một luồng hội thoại để giao tiếp
3. **Gửi Tin Nhắn**: Hỏi agent về sự khác biệt giữa Azure OpenAI và OpenAI
4. **Xử Lý Phê Duyệt Công Cụ**: Tự động phê duyệt các lần gọi công cụ MCP khi cần thiết
5. **Theo Dõi Thực Thi**: Giám sát tiến trình của agent và xử lý các hành động cần thiết
6. **Hiển Thị Kết Quả**: Hiển thị chi tiết cuộc hội thoại và việc sử dụng công cụ

**Tính năng không dùng khóa:**
- ✅ **Không có bí mật được mã hóa cứng** - Tất cả xác thực được xử lý bởi Azure identity
- ✅ **Bảo mật mặc định** - Sử dụng kiểm soát truy cập dựa trên vai trò
- ✅ **Triển khai đơn giản** - Không cần quản lý thông tin đăng nhập
- ✅ **Thân thiện với kiểm toán** - Tất cả truy cập được theo dõi thông qua Azure identity

Agent sẽ sử dụng các công cụ MCP để truy cập tài nguyên Microsoft Learn với đầy đủ bảo mật và không cần quản lý khóa API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
